In [1]:
import sys 
sys.path.append('..')
import warnings
warnings.filterwarnings('ignore')

In [4]:
from backend.sampler import MeshSampler
import trimesh
import numpy as np
import pandas as pd
import shutil
import os
from tqdm import tqdm_notebook as tqdm

In [5]:
import pybullet as pb

pybullet build time: Nov 28 2023 23:51:11


In [6]:
def rotate_pointcloud(pointcloud_A, rpy_BA=[np.pi / 2, 0, 0]):
    """
    The default rotation reflects the rotation used for the object during data collection.
    This calculates P_b, where P_b = R_b/a * P_a.
    R_b/a is rotation matrix of a wrt b frame.
    """
    # Rotate object
    rot_Q = pb.getQuaternionFromEuler(rpy_BA)
    rot_M = np.array(pb.getMatrixFromQuaternion(rot_Q)).reshape(3, 3)
    pointcloud_B = np.einsum('ij,kj->ki', rot_M, pointcloud_A)

    return pointcloud_B

In [7]:
def shapenet_rotate(mesh_original):
    '''In Shapenet, the front is the -Z axis with +Y still being the up axis. This function rotates the object to align with the canonical reference frame.
    Args:
        mesh_original: trimesh.Trimesh(), mesh from ShapeNet
    Returns:
        mesh: trimesh.Trimesh(), rotate mesh so that the front is the +X axis and +Y is the up axis.
    '''
    verts_original = np.array(mesh_original.vertices)

    rot_M = pb.getMatrixFromQuaternion(pb.getQuaternionFromEuler([np.pi/2, 0, -np.pi/2]))
    rot_M = np.array(rot_M).reshape(3, 3)
    verts = rotate_pointcloud(verts_original, [np.pi/2, 0, -np.pi/2])

    mesh = trimesh.Trimesh(vertices=verts, faces=mesh_original.faces)

    return mesh

In [8]:
import os
DATA_DIR = '../../../data/'
SAVE_DIR = '../../../data/'

categories = os.listdir(DATA_DIR)

#categories = [category for category in categories if os.path.isdir(os.path.join(DATA_DIR, category))]
#categories = ['vase', 'bed', 'tv', 'computer', 'lamp', 'pottedplant', 'couch', 'chair', 'desk', 'table']
#categories = ['bed', 'couch', 'vase', 'pottedplant']
categories = ['vase']

for category in categories:
    print(f'Processing {category}')
    category_data_dir = os.path.join(DATA_DIR, category)
    category_save_dir = os.path.join(SAVE_DIR, category)
    
    shapes = [shape for shape in os.listdir(category_data_dir) if os.path.isdir(os.path.join(category_data_dir, shape))]
    for shape in shapes:
        shape_data_dir = os.path.join(category_data_dir, shape)
        shape_save_dir = os.path.join(category_save_dir, shape)
        
        mesh_path = os.path.join(shape_data_dir, 'mesh_simplified.obj')
        print(f'Mesh at {mesh_path}')
        
        mesh = trimesh.load(mesh_path)
        
        rotated_mesh = shapenet_rotate(mesh)
        
        sampler = MeshSampler(mesh)
        
        sampler.compute_visible_faces()
        sampler.sample_points(n_points=100000)
        
        correct_mesh_points, correct_sdf, correct_normals = sampler.compute_sdf(sigma=0.0025)
        
        # Create positive and negative samples according to sdf values with their corresponding points
        # Positive samples are the points that are inside the mesh
        # Negative samples are the points that are outside the mesh
        positive_indices = correct_sdf > 0
        positive_samples = np.concatenate([correct_mesh_points[positive_indices], correct_sdf[positive_indices].reshape(-1, 1)], axis=1)
        
        negative_indices = correct_sdf < 0
        negative_samples = np.concatenate([correct_mesh_points[negative_indices], correct_sdf[negative_indices].reshape(-1, 1)], axis=1)
        
        #print(f'Positive samples shape: {positive_samples.shape}')
        #print(f'Negative samples shape: {negative_samples.shape}')
         
        np.savez(os.path.join(shape_save_dir, 'sdf.npz'), pos=positive_samples, neg=negative_samples)
        
'''
for bed_dir in bed_dirs:
    bed_data_dir = os.path.join(DATA_DIR, bed_dir)
    save_data_dir = os.path.join(SAVE_DIR, bed_dir)
    os.makedirs(save_data_dir, exist_ok=True)
    
    # If the sdf is there, skip
    if os.path.exists(os.path.join(save_data_dir, 'sdf.npz')):
        continue

    if os.path.exists(os.path.join(bed_data_dir, 'mesh_simplified.obj')):
        MESH = os.path.join(bed_data_dir, 'mesh_simplified.obj')
    print(MESH)

    sampler = MeshSampler(trimesh.load(MESH))
    sampler.compute_visible_faces()
    sampler.sample_points(n_points=25000)
    correct_mesh_points, correct_sdf, correct_normals = sampler.compute_sdf(sigma=0.0025)

    # Create positive and negative samples according to sdf values with their corresponding points
    # Positive samples are the points that are inside the mesh
    # Negative samples are the points that are outside the mesh
    positive_samples = np.concat(correct_mesh_points[correct_sdf > 0] , correct_sdf[correct_sdf > 0])
    negative_samples = np.concat(correct_mesh_points[correct_sdf < 0] , correct_sdf[correct_sdf < 0])

    # npz['pos'] = [x,y,z,sdf]  npz['neg'] = [x,y,z,sdf]

    # Save negative and positive samples with their corresponding sdf values
    np.savez(os.path.join(save_data_dir, 'sdf.npz'), pos=positive_samples, neg=negative_samples )


    # Save them to sdf.npz in the format of [points, sdf]
    # np.savez(os.path.join(save_data_dir, 'sdf.npz'), points=correct_mesh_points, sdf=correct_sdf)
'''

FileNotFoundError: [Errno 2] No such file or directory: '../../../data/'

# Create ShapeNet dataset

In [4]:
# for synsetId in tqdm(os.listdir(DATA_DIR)[33:]):
#     SYNSET_DIR = os.path.join(DATA_DIR, synsetId)
#     for i, modelId in enumerate(tqdm(os.listdir(SYNSET_DIR))):
#         if i == 5:
#             break
#         MODEL_DIR = os.path.join(SYNSET_DIR, modelId, 'models')
#         MESH = os.path.join(MODEL_DIR, 'model_normalized.obj')
        
#         sampler = MeshSampler(trimesh.load(MESH))
#         sampler.compute_visible_faces()
#         sampler.sample_points(n_points=250000)
#         correct_mesh_points, correct_sdf = sampler.compute_sdf(sigma=0.0025)
        
#         SAVE_PATH = os.path.join(SAVE_DIR, synsetId, modelId)
#         os.makedirs(SAVE_PATH, exist_ok=True)
#         np.save(os.path.join(SAVE_PATH, 'points.npy'), correct_mesh_points)
#         np.save(os.path.join(SAVE_PATH, 'sdfs.npy'), correct_sdf)

# Create ABC dataset

In [7]:
abc_dirs = os.listdir(DATA_DIR)
n_samples = 500


# from random import randint
# indices = []
# while len(indices) < 500:
#     i = randint(0, len(abc_dirs))
#     if i not in indices:
#         indices += [i]

In [8]:
# np.save('indices.npy', np.array(indices))

In [9]:
indices = np.load('indices.npy')

In [10]:
i = 388
while i < 500:
    print(i)
    index = indices[i]
    MESH = os.path.join(DATA_DIR, abc_dirs[index])

    sampler = MeshSampler(trimesh.load(MESH), check_faces=False)
    points = sampler.sample_points(n_points=500000)
    noisy_points, sdf, normals = sampler.compute_sdf(sigma=1.0)

    np.save(os.path.join(PCLOUD_DIR, str(i)+'.npy'), noisy_points)
    np.save(os.path.join(SDF_DIR, str(i)+'.npy'), sdf)
    np.save(os.path.join(NORMAL_DIR, str(i)+'.npy'), normals)
    np.save(os.path.join(ORIG_POINTS_DIR, str(i)+'.npy'), points)
    shutil.copy(MESH, os.path.join(MESH_DIR, str(i)+'.obj'))
    i += 1

388



389



390



393



394


...


495



496



497



498



499


In [15]:
data_pd = {'pclouds': [],
           'sdfs': [],
           'normals': [],
           'orig_points': [],
           'meshes': []}
for file in sorted(os.listdir(PCLOUD_DIR)):
    data_pd['pclouds'] += [os.path.join(PCLOUD_DIR, file)]
for file in sorted(os.listdir(SDF_DIR)):
    data_pd['sdfs'] += [os.path.join(SDF_DIR, file)]
for file in sorted(os.listdir(NORMAL_DIR)):
    data_pd['normals'] += [os.path.join(NORMAL_DIR, file)]
for file in sorted(os.listdir(ORIG_POINTS_DIR)):
    data_pd['orig_points'] += [os.path.join(ORIG_POINTS_DIR, file)]
for file in sorted(os.listdir(MESH_DIR)):
    data_pd['meshes'] += [os.path.join(MESH_DIR, file)]

df = pd.DataFrame.from_dict(data_pd)

In [22]:
df.iloc[0, 1]

'../../data/abc_data/sdfs/0.npy'

In [17]:
df.to_csv('../../data/abc_data_sigma_1.0.csv')